In [32]:
# Import libraries

import pandas as pd
import json
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from tqdm.autonotebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read JSON document with generated ids
with open('..\data\data-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [4]:
# Ground truth data

df_questions = pd.read_csv('..\data\ground-truth-retrieval.csv')
ground_truth = df_questions.to_dict(orient='records')

In [5]:
# Define model to use for embeddings
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [6]:
v = model.encode('symptoms of autism')
len(v)

384

In [7]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "question": {"type": "text"},
            "answer": {"type": "text"},
            "source": {"type": "text"},
            "focus_area": {"type": "text"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_focus_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_answer_focus_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "answer_focus_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "health-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'health-questions'})

In [8]:
for doc in tqdm(documents):
    question = doc['question']
    answer = doc['answer']
    focus = doc['focus_area']
    
    qa = question + ' ' + answer    
    qf = question + ' ' + focus
    af = answer + ' ' + focus
    qaf = answer + ' ' + answer + ' ' + focus

    doc['question_vector'] = model.encode(question)
    doc['answer_vector'] = model.encode(answer)
    doc['question_answer_vector'] = model.encode(qa)
    doc['question_focus_vector'] = model.encode(qf)
    doc['question_answer_focus_vector'] = model.encode(qaf)
    doc['answer_focus_vector'] = model.encode(af)

  0%|          | 0/292 [00:00<?, ?it/s]

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/292 [00:00<?, ?it/s]

In [26]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000}

    search_query = {
        "knn": knn,
        "_source": ["question", "answer", "source", "focus_area", "id", ""]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [22]:
# Hit rate
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

# mrr
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank]:  # If relevant document found
                total_score += 1 / (rank + 1)
                break

    return total_score / len(relevance_total)

In [12]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [35]:
def question_vector_knn(q):
    question = q['question']
    v_q = model.encode(question)
    return elastic_search_knn('question_vector', v_q)

# Evaluation
evaluate(ground_truth, question_vector_knn)

  0%|          | 0/1460 [00:00<?, ?it/s]

{'hit_rate': 0.963013698630137, 'mrr': 0.9348401826484016}

Combine all the KNN functions

In [28]:
def vector_knn(q, field):
    question = q['question']
    v_q = model.encode(question)
    return elastic_search_knn(field, v_q)

# Question only
def question_vector_knn(q):
    return vector_knn(q, 'question_vector')
    
# Answer only
def answer_vector_knn(q):
    return vector_knn(q, 'answer_vector')

# Question and Answer
def question_answer_vector_knn(q):
    return vector_knn(q, 'question_answer_vector')

# Question, Answer and Focus area
def question_answer_focus_vector_knn(q):
    return vector_knn(q, 'question_answer_focus_vector')

# Question and Focus area
def question_focus_vector_knn(q):
    return vector_knn(q, 'question_focus_vector')

# Answer and Focus area
def answer_focus_vector_knn(q):
    return vector_knn(q, 'answer_focus_vector')

In [29]:
def run_evaluations(ground_truth):
    results = {}

    results['question_vector'] = evaluate(ground_truth, question_vector_knn)
    results['answer_vector'] = evaluate(ground_truth, answer_vector_knn)
    results['question_answer_vector'] = evaluate(ground_truth, question_answer_vector_knn)
    results['question_answer_focus_vector'] = evaluate(ground_truth, question_answer_focus_vector_knn)
    results['question_focus_vector'] = evaluate(ground_truth, question_focus_vector_knn)
    results['answer_focus_vector'] = evaluate(ground_truth, answer_focus_vector_knn)
    
    return results

#### **Vector Evaluation Results**

In [30]:
def print_evaluation_results(results, sort_by='hit_rate'):
    sorted_results = sorted(
        results.items(), 
        key=lambda item: item[1][sort_by], 
        reverse=True
    )

    for knn_type, metrics in sorted_results:
        print(f"{knn_type}: Hit Rate: {metrics['hit_rate']:.4f}, MRR: {metrics['mrr']:.4f}")


results = run_evaluations(ground_truth)
# Print results sorted by 'hit_rate'
print("Evaluation Results Sorted by Hit Rate:")
print_evaluation_results(results, sort_by='hit_rate')

  0%|          | 0/1460 [00:00<?, ?it/s]

  0%|          | 0/1460 [00:00<?, ?it/s]

  0%|          | 0/1460 [00:00<?, ?it/s]

  0%|          | 0/1460 [00:00<?, ?it/s]

  0%|          | 0/1460 [00:00<?, ?it/s]

  0%|          | 0/1460 [00:00<?, ?it/s]

Evaluation Results Sorted by Hit Rate:
question_answer_vector: Hit Rate: 0.9849, MRR: 0.9551
answer_focus_vector: Hit Rate: 0.9760, MRR: 0.9333
question_answer_focus_vector: Hit Rate: 0.9705, MRR: 0.9136
question_vector: Hit Rate: 0.9630, MRR: 0.9348
question_focus_vector: Hit Rate: 0.9616, MRR: 0.9250
answer_vector: Hit Rate: 0.9575, MRR: 0.9026


The combination of question and answer vectors has the highest hit rate and mrr. Misearch, however, outperforms Elastic Search for both text and vector search.